In [17]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy

In [6]:
#!unzip -q /content/drive/MyDrive/hymenoptera_data.zip -d /content/

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [11]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.RandomResizedCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.Normalize(mean = [0.485,0.456,0.406],std = [0.229,0.224,0.225])
                                ])

In [12]:
# Create the ImageFolder dataset for the training set
train_dataset = datasets.ImageFolder(root=os.path.join("/content/hymenoptera_data", 'train'), transform=transform)

In [14]:
train_dataload = torch.utils.data.DataLoader(train_dataset, batch_size = 4, shuffle = True)

In [13]:
val_dataset = datasets.ImageFolder(root=os.path.join("/content/hymenoptera_data", 'val'), transform=transform)

In [15]:
val_dataload = torch.utils.data.DataLoader(val_dataset, batch_size = 4, shuffle = True)

In [16]:
train_dataset.classes

['ants', 'bees']

In [20]:
model = models.resnet18(pretrained = True)
num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, 2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.001)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 46.7MB/s]


In [21]:
num_epochs = 10

for epoch in range(num_epochs):
  for inputs,labels in train_dataload:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    loss = criterion(outputs,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f"Epoch: {epoch+1}, loss:{loss.item():.4f}")

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch: 1, loss:0.5350
Epoch: 2, loss:0.4510
Epoch: 3, loss:0.5011
Epoch: 4, loss:0.1380
Epoch: 5, loss:0.4392
Epoch: 6, loss:0.9192
Epoch: 7, loss:0.0951
Epoch: 8, loss:0.5874
Epoch: 9, loss:0.1835
Epoch: 10, loss:0.1228


In [22]:
correct = 0
total = 0

with torch.no_grad():
  for inputs, labels in train_dataload:
    inputs = inputs.to(device)
    labels = labels.to(device)

    _,predicted = torch.max(outputs,1)
    correct += (predicted==labels).sum().item()
    total += labels.size(0)

    Accuracy = correct / total

  print(f"Accuracy is {100 * Accuracy:.4f}%")

Accuracy is 49.5902%


In [23]:
correct = 0
total = 0

with torch.no_grad():
  for inputs, labels in val_dataload:
    inputs = inputs.to(device)
    labels = labels.to(device)

    _,predicted = torch.max(outputs,1)
    correct += (predicted==labels).sum().item()
    total += labels.size(0)

    Accuracy = correct / total

  print(f"Accuracy is {100 * Accuracy:.4f}%")

Accuracy is 45.7516%


In [24]:
model = models.resnet18(pretrained = True)
# Freeze all layers except the fully connected layers (classifier)
for param in model.parameters():   # here we froze starting layers and training just last layer
  param.requires_grad = False

num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, 2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.001)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [25]:
num_epochs = 20

for epoch in range(num_epochs):
  for inputs,labels in train_dataload:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    loss = criterion(outputs,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f"Epoch: {epoch+1}, loss:{loss.item():.4f}")

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch: 1, loss:0.6302
Epoch: 2, loss:0.5685
Epoch: 3, loss:0.7740
Epoch: 4, loss:0.4083
Epoch: 5, loss:0.6820
Epoch: 6, loss:0.1865
Epoch: 7, loss:0.1396
Epoch: 8, loss:0.1291
Epoch: 9, loss:0.8445
Epoch: 10, loss:0.1210
Epoch: 11, loss:0.1714
Epoch: 12, loss:0.4067
Epoch: 13, loss:0.0679
Epoch: 14, loss:0.4153
Epoch: 15, loss:0.2092
Epoch: 16, loss:0.4565
Epoch: 17, loss:0.2932
Epoch: 18, loss:0.3071
Epoch: 19, loss:0.2226
Epoch: 20, loss:0.1993


In [26]:
correct = 0
total = 0

with torch.no_grad():
  for inputs, labels in train_dataload:
    inputs = inputs.to(device)
    labels = labels.to(device)

    _,predicted = torch.max(outputs,1)
    correct += (predicted==labels).sum().item()
    total += labels.size(0)

    Accuracy = correct / total

  print(f"Accuracy is {100 * Accuracy:.4f}%")

Accuracy is 45.9016%


In [27]:
correct = 0
total = 0

with torch.no_grad():
  for inputs, labels in val_dataload:
    inputs = inputs.to(device)
    labels = labels.to(device)

    _,predicted = torch.max(outputs,1)
    correct += (predicted==labels).sum().item()
    total += labels.size(0)

    Accuracy = correct / total

  print(f"Accuracy is {100 * Accuracy:.4f}%")

Accuracy is 51.6340%
